Esse notebook tem o propósito de analisar todos os arquivos, juntar eles baseados no KNR de cada carro, remover as colunas que não há KNR correlacionado e separar essa coluna em pequenos arquivos divididos em tipo de carro

In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from functools import reduce

In [15]:
def read_excel_file(file_path):
    return pd.read_excel(file_path, header=1, usecols=lambda column: column not in ['Unnamed: 0'], engine='openpyxl')

In [47]:
excel_files = ['../static/FALHAS_04_05_06_2024.xlsx', '../static/RESULTADOS_02_03_2024.xlsx', '../static/RESULTADOS_04_06_2024.xlsx', '../static/RESULTADOS_06_2023_07_2023.xlsx', '../static/RESULTADOS_10_2023_11_2023.xlsx', '../static/RESULTADOS_12_2023_01_2024.xlsx', '../static/STATUS_PREDICTOR_ANO.xlsx']

In [51]:
dataframes = [read_excel_file(file) for file in excel_files]

c:\Users\mario\OneDrive\Documentos\2024-2A-T08-EC07-G04\src\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\mario\OneDrive\Documentos\2024-2A-T08-EC07-G04\src\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\mario\OneDrive\Documentos\2024-2A-T08-EC07-G04\src\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\mario\OneDrive\Documentos\2024-2A-T08-EC07-G04\src\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook co

In [37]:
teste = pd.read_excel("../static/RESULTADOS_08_2023_09_2023.xlsx", header=3, usecols=lambda column: column not in ['Unnamed: 0'], engine='openpyxl')

teste

c:\Users\mario\OneDrive\Documentos\2024-2A-T08-EC07-G04\src\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA
0,2023-2316418,012001,1,10,NaN,0,NaN,2023-08-02 12:04:16
1,2023-2316418,012002,1,10,NaN,0,NaN,2023-08-02 12:18:14
2,2023-2316418,012003,1,10,NaN,0,NaN,2023-08-02 12:18:27
3,2023-2316418,012004,1,10,NaN,0,NaN,2023-08-02 12:17:27
4,2023-2316418,018001,1,10,NaN,0,NaN,2023-08-02 11:57:29
...,...,...,...,...,...,...,...,...
1048566,2023-3016074,140700,2,10,Nm,2,8.05 ...,2023-08-07 07:24:17
1048567,2023-3016074,140700,2,10,Nm,0,8.10 ...,2023-08-07 07:24:17
1048568,2023-3016074,160100,2,10,Grad,1,129.00 ...,2023-08-07 06:56:45
1048569,2023-3016074,160100,2,10,Nm,0,14.10 ...,2023-08-07 06:56:45


In [49]:
teste2 = pd.read_excel('../static/FALHAS_PREDICT.xlsx', header=2, usecols=lambda column: column not in ['Unnamed: 0'], engine='openpyxl')

teste2

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,DATA
0,2023-1326197,T-CROSS,0Q0Q,DHS,PCEQS100,2043521,ZP5,Porta dianteira Folga (Ajuste),2023-01-09 22:24:44.272
1,2023-1326197,T-CROSS,0Q0Q,DHS,PCEQS100,2043521,ZP5,Porta traseira Folga (Ajuste),2023-01-09 22:25:02.960
2,2023-4130482,AUDI Q3,2Y2Y,DSN,PCEQS999,2054701,ESPC,Lanterna tras (09) Interfere,2023-01-10 09:18:25.238
3,2023-4130482,AUDI Q3,2Y2Y,DSN,PCEQS999,2054701,ESPC,Emblema / logo - Adesivos em geral Solto(a) ¨,2023-01-10 09:18:40.728
4,2023-4130664,AUDI Q3,6Y6Y,DSN,PCEQS805,7000240,ZP82,Painel instrumentos Ruído (-),2023-01-10 11:17:56.489
...,...,...,...,...,...,...,...,...,...
663814,2024-1336453,T-CROSS,K2K2,DHS,HCEQS506,2051427,ZP7,TAMPA TRASEIRA 05. SUJEIRA NA PINTURA,2024-03-27 14:52:39.400
663815,2024-1336485,T-CROSS,0Q0Q,DHS,HCEQS506,2044862,ZP7,PORTA TRASEIRA 03. CAVACO,2024-03-27 14:53:03.187
663816,2024-1346004,T-CROSS,A1A1,DHS,HCEQS506,2052733,ZP7,RACK DO TETO FELTRO APARENTE,2024-03-27 14:53:25.992
663817,2024-1336453,T-CROSS,K2K2,DHS,HCEQS506,2051427,ZP7,TAMPA TRASEIRA 07. FERVIDO,2024-03-27 14:53:29.873


In [52]:
print(dataframes)

[                 KNR   MODELO   COR MOTOR   ESTACAO  USUARIO HALLE  \
0       2023-2056234  T-CROSS  2R2R   DHS  HCEQS506  4013334  ZP7    
1       2023-2056234  T-CROSS  2R2R   DHS  HCEQS506  4013334  ZP7    
2       2023-2056234  T-CROSS  2R2R   DHS  PCEQS620  2058570  ZP6    
3       2023-2056234  T-CROSS  2R2R   DHS  PCEQS999  2087715  ESPC   
4       2023-3896001  T-CROSS  0Q0Q        HCEQS203  2043971  ZP5A   
...              ...      ...   ...   ...       ...      ...   ...   
328896  2024-2976012  T-CROSS  0QA1   DHS  PCEQS315  4005452  ZP5A   
328897  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS101  4006390  ZP5    
328898  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS101  4006390  ZP5    
328899  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS118  2075628         
328900  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS314  2061350  ZP5A   

                                         FALHA                    DATA  
0              PARALAMA DIANTEIRO 01. AMASSADO 2024-05-20 21:31:51.992  
1           

In [53]:
dataframes.append(teste)

In [54]:
dataframes.append(teste2)

In [55]:
dataframes

[                 KNR   MODELO   COR MOTOR   ESTACAO  USUARIO HALLE  \
 0       2023-2056234  T-CROSS  2R2R   DHS  HCEQS506  4013334  ZP7    
 1       2023-2056234  T-CROSS  2R2R   DHS  HCEQS506  4013334  ZP7    
 2       2023-2056234  T-CROSS  2R2R   DHS  PCEQS620  2058570  ZP6    
 3       2023-2056234  T-CROSS  2R2R   DHS  PCEQS999  2087715  ESPC   
 4       2023-3896001  T-CROSS  0Q0Q        HCEQS203  2043971  ZP5A   
 ...              ...      ...   ...   ...       ...      ...   ...   
 328896  2024-2976012  T-CROSS  0QA1   DHS  PCEQS315  4005452  ZP5A   
 328897  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS101  4006390  ZP5    
 328898  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS101  4006390  ZP5    
 328899  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS118  2075628         
 328900  2024-2976013  T-CROSS  0Q0Q   DHS  PCEQS314  2061350  ZP5A   
 
                                          FALHA                    DATA  
 0              PARALAMA DIANTEIRO 01. AMASSADO 2024-05-20 21:31:51.992 

In [56]:
for df in dataframes:
    if 'KNR' not in df.columns:
        raise KeyError(f"'KNR' column not found in one of the DataFrames: {df.columns.tolist()}")

In [ ]:
col = ""
df = df.astype(str)
df[col] = df.groupby(["KNR"])[col].transform(lambda x: "; ".join(x)) 

In [60]:
def merge_with_suffixes(left, right):
    common_cols = set(left.columns).intersection(set(right.columns)) - {'KNR'}
    if common_cols:
        right = right.rename(columns={col: f"{col}_right" for col in common_cols})
    return pd.merge(left, right, on='KNR')

In [62]:
merged_df = reduce(lambda left, right: pd.merge(left, right, on='KNR'), dataframes)

MergeError: Passing 'suffixes' which cause duplicate columns {'DATA_x'} is not allowed.